In [122]:
import numpy as np
import copy
from sklearn.neural_network import MLPRegressor
from numpy.random import *
from operator import itemgetter

# 1.InputFieldData

In [445]:
def sortPosInfo(pos_list_with_num,info_name):
    np_pos_list_with_num = np.asarray(pos_list_with_num)
    sorted_np_pos_list_with_num = np.array(sorted(np_pos_list_with_num,key=itemgetter(1)))
    pos_list = []
    for pos_info in sorted_np_pos_list_with_num:
        pos_list.append(pos_info[0])
    print("{0} = {1}".format(info_name,pos_list))
    return pos_list

In [448]:
def inputFieldData(text_path):
    f = open(text_path)
    line = f.readline()
    line_count = 1
    field_size = int(len(line) / 2)
    field = np.empty((0,field_size),int)
    agents_pos_list = []
    goal_pos_list = []
    while line:
        last_idx = line.find("\n")
        field_row = np.array([])
        for i in range(0,last_idx,2):
            field_row = np.append(field_row,int(line[i]))
            if int(line[i]) % 2 == 0 and int(line[i]) != 0:
                agent_data = np.array([np.array([line_count-1,int(i/2)]),int(line[i])])
                agents_pos_list.append(agent_data)
            if int(line[i]) % 2 == 1 and int(line[i]) != 1:
                goal_data = np.array([np.array([line_count-1,int(i/2)]),int(line[i])])
                goal_pos_list.append(goal_data)
        field = np.append(field,np.array([field_row]),axis=0)
        line = f.readline()
        line_count += 1
    f.close()
    print("init field = \n {0}".format(field))
    agents_pos_list = sortPosInfo(agents_pos_list,"agents_pos_list")
    goal_pos_list = sortPosInfo(goal_pos_list,"goal_pos_list")
    return field,agents_pos_list,goal_pos_list

In [481]:
init_field,agents_pos_list,goal_pos_list = inputFieldData("test.txt")
field_row = len(init_field[0])
field_col = len(init_field)

init field = 
 [[ 1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  4.  0.  2.  0.  0.  1.]
 [ 1.  0.  1.  7.  1.  3.  0.  0.  1.]
 [ 1.  0.  1.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  6.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  5.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.]]
agents_pos_list = [array([1, 5]), array([1, 3]), array([4, 3])]
goal_pos_list = [array([2, 5]), array([5, 6]), array([2, 3])]


# 2.DecideActions

In [482]:
def moveToNextPos(pos,action):
    if action == 0:
        next_pos = [pos[0]-1,pos[1]]
    elif action == 1:
        next_pos = [pos[0],pos[1]+1]
    elif action == 2:
        next_pos = [pos[0]+1,pos[1]]
    elif action== 3:
        next_pos = [pos[0],pos[1]-1]
    else:
        next_pos = [pos[0],pos[1]]
    return next_pos

In [483]:
def buildNetwork(X_train,y_train):
    clf = MLPRegressor(solver="adam",random_state=1,max_iter=1000,hidden_layer_sizes=(20,50,10))
    print("X_train.shape = {0} , y_train.shape = {1}".format(X_train.shape,y_train.shape))
    clf.fit(X_train,y_train)
    return clf

In [635]:
def checkNextField(poses,next_poses,field,agent_num):
    next_pos = next_poses[agent_num]
    action_flag = True
    #行動先が行動可能かどうか判定
    if field[next_pos[0],next_pos[1]] != 1: #行動先が壁でないとき
        for j in range(len(next_poses)):
            #行動先に先約があるとき
            if agent_num == j: continue
            if next_poses[agent_num] == next_poses[j]:
                action_flag = False
            #他エージェントと交差するとき
            if next_poses[agent_num] == poses[j] and next_poses[j] == poses[agent_num]:
                action_flag = False
    else:
        action_flag = False
    return action_flag

In [636]:
def checkAchievingGoal(poses,goal_flags):
    for i in range(len(poses)):
        if poses[i] == goal_pos_list[i].tolist():
            goal_flags[i] = True
    return goal_flags

In [661]:
def checkActions(pred_actions,field,poses,goal_flags):
    next_poses = []
    for i in range(len(pred_actions)):
        next_poses.append(moveToNextPos(poses[i],pred_actions[i]))
        goal_flags = checkAchievingGoal(poses,goal_flags)
        if goal_flags[i]:
            pred_actions[i] = 4
            next_poses[i] = moveToNextPos(poses[i],pred_actions[i])
    for i in range(len(next_poses)):
        if goal_flags[i]:
            continue
        #行動先が行動可能かどうか判定
        action_flag = checkNextField(poses,next_poses,field,i)
        #行動不可能なとき，注目するエージェントの行動先を変更
        cnt = 0
        while action_flag == False:
            pred_actions[i] = int(pred_actions[i] + rand() * 3 + 1) % 4
            next_poses[i] = moveToNextPos(poses[i],pred_actions[i])
            action_flag = checkNextField(poses,next_poses,field,i)
            cnt += 1
            if cnt > 10000:
                pred_actions[i] = 4
                break
    goal_flags = checkAchievingGoal(poses,goal_flags)
    for i in range(len(goal_flags)):
        if goal_flags[i]:
            pred_actions[i] = 4 #停止
    return pred_actions

In [662]:
def predictActions(clf,field):
    X_test = np.asarray(field).reshape(1,field_row*field_col)
    actions = clf.predict(X_test)[0]
    for i in range(len(actions)):
        actions[i] = int(actions[i])
        
    if rand() < 0.3:
        for i in range(len(actions)):
            actions[i] = int(actions[i] + rand() * 3 + 1) % 4
    return actions

In [663]:
#特殊地点：エージェントのゴール地点
def updateField(poses,next_poses,field):
    for i in range(len(agents_pos_list)):
        pos = poses[i]
        next_pos = next_poses[i]
        #エージェントがいずれかの方向に進行する場合
        if pos != next_pos:
            #現在いる地点が特殊地点でないとき
            if field[pos[0],pos[1]] % 2 == 0:
                field[pos[0],pos[1]] = 0
            #現在いる地点が特殊地点のときは現在地の値は変更しない
            
            #遷移先が特殊地点でないとき
            if field[next_pos[0],next_pos[1]] % 2 == 0:
                field[next_pos[0],next_pos[1]] = i*2 + 2
            #遷移先が特殊地点のときは遷移先の値は変更しない

In [664]:
def printFieldData(field_data,action_data):
    for i in range(len(field_data)):
        print(field_data[i])
        print(action_data[i])
    print("--------------------------------------------------")

In [700]:
def createTrainData(init_field,init_poses,clf,path_count,best_field_data,best_action_data):
    field_data = [copy.deepcopy(init_field)]
    field = copy.deepcopy(init_field)
    poses = init_poses
    action_data = []
    goal_flags = [False for i in range(len(poses))]
    create_flag = False
    for i in range(FIND_ROOT_MAX_LOOP):
        if clf != None:
            pred_actions = predictActions(clf,field)
        else:
            pred_actions = [int(rand()*4) for i in range(len(agents_pos_list))]
        #予測したアクションに対してチェックを行い，壁や他エージェントにぶつかる挙動を変更する
        actions = checkActions(pred_actions,field,poses,goal_flags)
        #タイムステップ毎のアクションを保持するaction_dataの末尾に次の行動を追加
        action_data.append(actions)
        #actionsを元に次のfieldへと遷移させる
        next_poses = []
        for j in range(len(agents_pos_list)):
            next_poses.append(moveToNextPos(poses[j],actions[j]))
        updateField(poses,next_poses,field)
        field_data.append(copy.deepcopy(field))
        poses = next_poses
        if goal_flags[0] and goal_flags[1] and goal_flags[2]:
            action_data.append([4,4,4])
            np_field_data = np.asarray(field_data)
            np_action_data = np.asarray(action_data)
            if path_count > np_field_data.shape[0]:
                path_count = np_field_data.shape[0]
                clf = buildNetwork(np_field_data.reshape(np_field_data.shape[0],np_field_data.shape[1]*np_field_data.shape[2]),np_action_data)
#                 printFieldData(field_data,action_data)
                best_field_data = copy.deepcopy(field_data)
                best_action_data = copy.deepcopy(action_data)
                create_flag = True
            break
#     for i in range(30):
#         print(field_data[i])
#         print("action = {0} \n".format(action_data[i]))
#     print("--------------------------------------------------")
    return  clf,path_count,best_field_data,best_action_data,create_flag

In [703]:
FIND_ROOT_MAX_LOOP = 100
clf = None
path_count = 1e+7
agents_poses = []
best_field_data = []
best_action_data = []
for agents_pos in agents_pos_list:
    agents_poses.append(agents_pos.tolist())
create_flag = False
cnt = 0
for i in range(2000):
    if create_flag == False:
        cnt += 1
    else:
        cnt = 0
    if cnt > 300:
        break
    init_poses = agents_poses
    if (i+1) % 100 == 0:
        print("epoch = {0}".format(i+1))
    clf,path_count,best_field_data,best_action_data,create_flag = createTrainData(init_field,init_poses,clf,path_count,best_field_data,best_action_data)

X_train.shape = (94, 63) , y_train.shape = (94, 3)
X_train.shape = (71, 63) , y_train.shape = (71, 3)
X_train.shape = (57, 63) , y_train.shape = (57, 3)
X_train.shape = (45, 63) , y_train.shape = (45, 3)
X_train.shape = (23, 63) , y_train.shape = (23, 3)
X_train.shape = (17, 63) , y_train.shape = (17, 3)
X_train.shape = (13, 63) , y_train.shape = (13, 3)
X_train.shape = (11, 63) , y_train.shape = (11, 3)
epoch = 100
epoch = 200
epoch = 300


In [704]:
printFieldData(best_field_data,best_action_data)

[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  4.  0.  2.  0.  0.  1.]
 [ 1.  0.  1.  7.  1.  3.  0.  0.  1.]
 [ 1.  0.  1.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  6.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  5.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.]]
[ 1.  1.  0.]
[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  4.  0.  2.  0.  1.]
 [ 1.  0.  1.  7.  1.  3.  0.  0.  1.]
 [ 1.  0.  1.  6.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  5.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.]]
[ 2.  1.  0.]
[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  4.  0.  0.  1.]
 [ 1.  0.  1.  7.  1.  3.  2.  0.  1.]
 [ 1.  0.  1.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  5.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.]]
[ 4.  2.  4.]
[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  1.  7.  1.  3.  2.  0.  1.]
 [ 1.  0.  1.  0.  